In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

from tensorflow import keras
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

# **Tabla de contenidos**
***
Redes Neuronales (MLP)
1.  Modelo 1 (baseline)

    1.1. Creación del modelo baseline

    1.2. Evaluación del modelo baseline

2.  Modelo 2

    2.1. Creación del modelo 2

    2.2. Evaluación del modelo 2
    
3.  Modelo 3

    3.1. Creación del modelo 3

    3.2. Evaluación del modelo 3

4.  Evaluación general y conclusiones

***

# **Redes Neuronales** (MLP)

Un **modelo MLP** *(perceptrón multicapa)* es una clase de red neuronal que se caracteriza por estar inspirado en las neuronas biológicas y su principal cometido es la **clasificación de datos**.

En nuestro caso vamos a utilizar un modelo MLP para predecir y clasificar **qué tipo de crimen es más posible que le ocurra a una víctima** dependiendo de los siguientes datos:

In [12]:
ruta_csv = '../../../data/Personal_Victimization_Original.csv'
datos = pd.read_csv(ruta_csv)
datos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 68852 entries, 0 to 68851
Data columns (total 37 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   idper           68852 non-null  object 
 1   yearq           68852 non-null  float64
 2   year            68852 non-null  int64  
 3   ager            68852 non-null  int64  
 4   sex             68852 non-null  int64  
 5   hispanic        68852 non-null  int64  
 6   race            68852 non-null  int64  
 7   race_ethnicity  68852 non-null  int64  
 8   hincome1        68852 non-null  int64  
 9   hincome2        68852 non-null  int64  
 10  marital         68852 non-null  int64  
 11  popsize         68852 non-null  int64  
 12  region          68852 non-null  int64  
 13  msa             68852 non-null  int64  
 14  locality        68852 non-null  int64  
 15  educatn1        68852 non-null  int64  
 16  educatn2        68852 non-null  int64  
 17  veteran         68852 non-null 

En este caso la variable a predecir seria ***newoff***.

In [ ]:
print(datos["newoff"])
print("-------------------")
print(datos["newoff"].value_counts())

0        5
1        2
2        4
3        4
4        4
        ..
68847    4
68848    4
68849    4
68850    4
68851    4
Name: newoff, Length: 68852, dtype: int64
newoff
4    42657
3    13250
2     7199
1     3583
5     2163
Name: count, dtype: int64


## 1. Explicación modelo baseline

### 1.1. Creación modelo baseline


0        5
1        2
2        4
3        4
4        4
        ..
68847    4
68848    4
68849    4
68850    4
68851    4
Name: newoff, Length: 68852, dtype: int64

In [ ]:
# Modelo simple
model_baseline = Sequential([
    Dense(32, activation='relu', input_shape=(16,)),
    Dense(7, activation='softmax')  # 7 clases
])

In [ ]:
model_baseline.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])

In [ ]:
history_crime = model_baseline.fit(X_train_scaled, y_train,
                                   validation_split=0.2,
                                   epochs=20,
                                   batch_size=32,
                                   verbose=1)

### 1.2. Evaluación modelo baseline

In [ ]:
# Evaluar train

train_loss, train_acc = model_baseline.evaluate(X_train_scaled, y_train, verbose=0)

print(f"Train Accuracy: {train_acc:.4f} ({train_acc*100:.2f}%)")
print(f"Train Loss: {train_loss:.4f}")
print("="*50)

print("\n Baseline del azar (3 clases): 33.3%")

In [ ]:
# Evaluar test

test_loss, test_acc = model_baseline.evaluate(X_test_scaled, y_test, verbose=0)

print("="*50)
print(" RESULTADOS BASELINE - Tipos crimen")
print("="*50)
print(f"Test Accuracy: {test_acc:.4f} ({test_acc*100:.2f}%)")
print(f"Test Loss: {test_loss:.4f}")
print("="*50)

In [ ]:
# Visualización de curvas de entrenamiento
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 4))

# Crear rango de epochs (1 a 20, no 0 a 19)
epochs = range(1, len(history_crime.history['loss']) + 1)

# Loss
ax1.plot(epochs, history_crime.history['loss'], label='Train Loss', linewidth=2)
ax1.plot(epochs, history_crime.history['val_loss'], label='Val Loss', linewidth=2)
ax1.set_xlabel('Epoch')
ax1.set_ylabel('Loss')
ax1.set_title('Curvas de Loss')
ax1.legend()
ax1.grid(alpha=0.3)

# Accuracy
ax2.plot(epochs, history_crime.history['accuracy'], label='Train Accuracy', linewidth=2)
ax2.plot(epochs, history_crime.history['val_accuracy'], label='Val Accuracy', linewidth=2)
ax2.set_xlabel('Epoch')
ax2.set_ylabel('Accuracy')
ax2.set_title('Curvas de Accuracy')
ax2.legend()
ax2.grid(alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# Matriz de confusión
y_pred_crime = model_baseline.predict(X_test_scaled)
y_pred_crime_classes = np.argmax(y_pred_crime, axis=1)

cm = confusion_matrix(y_test, y_pred_crime_classes)

plt.figure(figsize=(6, 5))
sns.heatmap(cm, annot=True, fmt='d', cmap='Reds', 
            xticklabels=['Dermason', 'Sira', 'Seker', 'Horoz', 'Cali', 'Barbunya', 'Bombay'],
            yticklabels=['Dermason', 'Sira', 'Seker', 'Horoz', 'Cali', 'Barbunya', 'Bombay'])
plt.title('Matriz de Confusión - Tipos judía')
plt.ylabel('Real')
plt.xlabel('Predicción')
plt.show()

print("\n Classification Report:")
print(classification_report(y_test, y_pred_crime_classes, 
                          target_names=['Dermason', 'Sira', 'Seker', 'Horoz', 'Cali', 'Barbunya', 'Bombay']))

## 2. Explicación modelo 2

### 2.1. Creación modelo 2

### 2.2. Evaluación modelo 2

## 3. Explicación modelo 3

### 3.1. Creación modelo 3

### 3.2. Evaluación modelo 3

# Evaluación general de todos los modelos